In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [8]:
from transformers import AutoTokenizer, LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")


In [3]:
prompt = "Generate a counter-argument for the tweet: Vaccines are bad for health"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=250)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


"Generate a counter-argument for the tweet: Vaccines are bad for health, they are a government-controlled system.\nTweet: Vaccines are bad for health, they are a government-controlled system.\nCounter-argument:\nWhile it's true that vaccines are heavily regulated and monitored, this doesn't necessarily mean they are bad for health. In fact, vaccination has been shown to be a highly effective way to prevent the spread of infectious diseases. By getting vaccinated, individuals are protecting not only themselves, but also those around them, including vulnerable populations such as the elderly and young children. Additionally, the benefits of vaccination far outweigh any perceived risks. According to the World Health Organization, vaccines have saved millions of lives worldwide and have been instrumental in controlling the spread of diseases such as polio, measles, and influenza. Furthermore, vaccination is not a government-controlled system, but rather a collaborative effort between healt

In [9]:
def create_message_column_without_labels(row):
        # Initialize an empty list to store the messages.
    messages = []
    
    # Create a 'user' message dictionary with 'content' and 'role' keys.
#     print(row['text'])
    user = {
        "content": f"Generate Counter Argument for the anti-vaccine tweet:\n Tweet: {row['text']}\n##Output: ",
        "role": "user"
    }
    
    # Append the 'user' message to the 'messages' list.
    messages.append(user)
    
    # Create an 'assistant' message dictionary with 'content' and 'role' keys.
    assistant = {
        "content": f"{row['counter_argument']}",
        "role": "assistant"
    }
    
    # Append the 'assistant' message to the 'messages' list.
    messages.append(assistant)
    
    # Return a dictionary with a 'messages' key and the 'messages' list as its value.
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}

In [10]:
# LABEL DESCRIPTIONS

labels_map = {}
labels_map['religious'] = "religious beliefs and their influence on views about vaccines"
labels_map['political'] = "the political factors that affect perceptions of vaccine use"
labels_map['ingredients'] = "concerns about the ingredients and chemical components in vaccines"
labels_map['unnecessary'] = "the importance and necessity of getting vaccinated to prevent diseases"
labels_map['conspiracy'] = "conspiracy theories suggesting hidden motives behind vaccination efforts"
labels_map['mandatory'] = "the debate over personal choice versus mandates in vaccination policies"
labels_map['ineffective'] = "evidence and reasons that support the effectiveness of vaccines"
labels_map['side-effect'] = "potential side effects and adverse reactions associated with vaccines"
labels_map['pharma'] = "the role of pharmaceutical companies and concerns about profit motives"
labels_map['rushed'] = "claims that vaccines were approved or developed without sufficient testing"
labels_map['country'] = "national biases and objections to vaccines produced by specific countries"

In [11]:
def create_message_column_with_labels(row):
        # Initialize an empty list to store the messages.
    messages = []
    # print(row)
    prompt= f"Generate Counter Argument for the anti-vaccine tweet:\n Tweet: {row['text']}\n Talk About "

    lab=row['labels'].split()
    if isinstance(lab, list):
            mapped_labels = " and ".join([labels_map.get(l, "") for l in lab])
    else:
            mapped_labels = labels_map.get(lab, "")
    # print(mapped_labels)
    prompt+= mapped_labels
    prompt+= " ##Output:"
    user = {
        "content": prompt,
        "role": "user"
    }
    
    # Append the 'user' message to the 'messages' list.
    messages.append(user)
    
    # Create an 'assistant' message dictionary with 'content' and 'role' keys.
    assistant = {
        "content": f"{row['counter_argument']}",
        "role": "assistant"
    }
    
    # Append the 'assistant' message to the 'messages' list.
    messages.append(assistant)
    
    # Return a dictionary with a 'messages' key and the 'messages' list as its value.
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}


In [12]:
from datasets import load_dataset
train_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/train_CA_with_label_desc.csv',split='train')
val_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/val_data_with_labels_desc.csv',split='train')

# train_data=train_data[:5]
dataset_chatml = train_data.map(create_message_column_with_labels)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)

In [5]:
from datasets import load_dataset
train_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/train_CA_without_labels.csv',split='train')
val_data= load_dataset('csv', data_files='/home/sohampoddar/HDD2/utsav/Data/val_CA_without_labels.csv',split='train')

In [6]:
dataset_chatml = train_data.map(create_message_column_with_labels)

dataset_chatml = dataset_chatml.map(format_dataset_chatml)
                            
val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)

In [13]:
import torch
import peft ,trl

# 'load_dataset' is a function from the 'datasets' library by Hugging Face which allows you to load a dataset.
from datasets import load_dataset

from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed
)

# 'SFTTrainer' is a class from the 'trl' library that provides a trainer for soft fine-tuning.
from trl import SFTTrainer

In [17]:
model_id="meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [14]:

# 'torch.cuda.is_bf16_supported()' is a function that checks if bfloat16 is supported on the current CUDA device.
# If bfloat16 is supported, 'compute_dtype' is set to 'torch.bfloat16' and 'attn_implementation' is set to 'flash_attention_2'.
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
# If bfloat16 is not supported, 'compute_dtype' is set to 'torch.float16' and 'attn_implementation' is set to 'sdpa'.
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

# # This line of code is used to print the value of 'attn_implementation', which indicates the chosen attention implementation.
print(attn_implementation)

flash_attention_2


In [18]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding token if none is defined

# Set the pad_token_id
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
# Set padding side if needed
tokenizer.padding_side = 'right'
# set cuda available device
if torch.cuda.is_available():
    device = torch.device("cuda")  # 'cuda' will point to the visible GPU (e.g., GPU 0 as set by CUDA_VISIBLE_DEVICES)
    print("Using GPU:", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("Using CPU")

use_4bit = True
# 'bnb_4bit_compute_dtype' is the data type that should be used for computations with the 4-bit base model. In this case, it is set to 'bfloat16'.
bnb_4bit_compute_dtype = "bfloat16"

# 'bnb_4bit_quant_type' is the type of quantization that should be used for the 4-bit base model. In this case, it is set to 'nf4'.
bnb_4bit_quant_type = "nf4"

# 'use_double_quant' is a boolean that controls whether nested quantization should be used for the 4-bit base model.
use_double_quant = True
bnb_config= BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

model = AutoModelForCausalLM.from_pretrained(
          model_id, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map="auto",
#           attn_implementation=attn_implementation
)
model.gradient_checkpointing_enable()  # reduce number of stored activations
model = prepare_model_for_kbit_training(model)
target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
peft_config = LoraConfig(
        r=4,
        lora_alpha=32,
        lora_dropout=0.1,
        task_type=TaskType.CAUSAL_LM,
        target_modules=target_modules,
)
# model = get_peft_model(model, peft_config)


Using GPU: NVIDIA RTX A5000


In [11]:

args = TrainingArguments(
        output_dir="/home/sohampoddar/HDD2/utsav/Cross_Experiment_2/Models/llama-3.2-fine-tuned_with_labels",
        evaluation_strategy="epoch",
        do_eval=True,
        optim="adamw_torch",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=2,
        log_level="debug",
        save_strategy="epoch",
        learning_rate=1e-4,
        logging_dir=None,  # Directory for storing logs
        logging_steps=100,     # Log after every 100 steps
        logging_strategy="steps",  # Log by steps instead of epochs
        fp16=True,
#         bf16 = torch.cuda.is_bf16_supported(),
        eval_steps=100,
        num_train_epochs=2,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
#     report_to="wandb",
        seed=42,
)


/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
import os
from rouge_score import rouge_scorer
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
print(torch.cuda.current_device())

response_template = "assistant<|end_header_id|>"
instruction_template = "user<|end_header_id|>"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template,response_template=response_template, tokenizer=tokenizer)



        # Round the values for better readability
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_chatml,
        eval_dataset=val_dataset_chatml,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
#         compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        args=args,
        data_collator=collator
)
trainer.train()

# 'trainer.save_model()' is a method that saves the trained model locally.
# The model will be saved in the directory specified by 'output_dir' in the training arguments.
trainer.save_model()


0


/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/trl/trainer/utils.py:162: UserWarning: The pad_token_id and eos_token_id values of this tokenizer are identical. If you are planning for multi-turn training, it can result in the model continuously generating questions and answers without eos token. To avoid this, set the pad_token_id to a different value.
  warnings.warn(
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/transformers/training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eva

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/sohampoddar/anaconda3/envs/dl/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.190700,1.132149
2,0.933200,1.108074


[codecarbon INFO @ 00:22:44] Energy consumed for RAM : 0.000196 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 00:22:44] Energy consumed for all GPUs : 0.001038 kWh. Total GPU Power : 249.07603937658658 W
[codecarbon INFO @ 00:22:44] Energy consumed for all CPUs : 0.000229 kWh. Total CPU Power : 54.97827430757787 W
[codecarbon INFO @ 00:22:44] 0.001464 kWh of electricity used since the beginning.
[codecarbon INFO @ 00:22:59] Energy consumed for RAM : 0.000392 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 00:22:59] Energy consumed for all GPUs : 0.002114 kWh. Total GPU Power : 258.30156356479944 W
[codecarbon INFO @ 00:22:59] Energy consumed for all CPUs : 0.000458 kWh. Total CPU Power : 54.79669335272753 W
[codecarbon INFO @ 00:22:59] 0.002964 kWh of electricity used since the beginning.
[codecarbon INFO @ 00:23:14] Energy consumed for RAM : 0.000588 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 00:23:14] Energy consumed for all GPUs : 0.003197 kWh. Tota

In [36]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig
model_path = "/home/sohampoddar/HDD2/utsav/Model_Codes_with_labels/phi-3-mini-4k-instruct"
tokenizer_path = "/home/sohampoddar/HDD2/utsav/Model_Codes_with_labels/phi-3-mini-4k-instruct"
# connect to device
if torch.cuda.is_available():
    device = torch.device("cuda")  # 'cuda' will point to the visible GPU (e.g., GPU 0 as set by CUDA_VISIBLE_DEVICES)
    print("Using GPU:", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("Using CPU")

compute_dtype = torch.float16
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

Using GPU: NVIDIA RTX A5000


Loading checkpoint shards: 100%|██████████| 2/2 [00:33<00:00, 16.74s/it]


In [27]:
import evaluate
from evaluate import load  # Correct import for metrics
metric = evaluate.load("rouge")

In [37]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [29]:
val_dataset_chatml=val_data.map(create_message_column_with_labels)
val_dataset_chatml=val_dataset_chatml.map(format_dataset_chatml)
dataset_chatml = val_dataset_chatml.train_test_split(test_size=0.05)
dataset_chatml

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'counter_argument', 'messages'],
        num_rows: 937
    })
    test: Dataset({
        features: ['labels', 'text', 'counter_argument', 'messages'],
        num_rows: 50
    })
})

In [30]:
def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=100, do_sample=True, num_beams=1, temperature=0.7, top_k=50, top_p=0.9,repetition_penalty=1.2, max_time= 50)
    # print(outputs[0]['generated_text'])
    return outputs[0]['generated_text'][len(prompt):].strip()

In [19]:
test_inference(dataset_chatml['test'][1]['messages'][0]['content'])

"While it is important to prioritize public health and safety, there are no known or proven loopholes that would allow individuals to avoid getting vaccinated against COVID-19. The vaccines have undergone rigorous testing and approval processes by regulatory agencies such as FDA before being approved for use in emergency situations like this pandemic. It's essential to trust scientific guidance from healthcare professionals rather than seeking out potentially harmful alternatives. Vaccination plays a crucial role in controlling the spread of infectious diseases and protecting vulnerable populations. Ignoring vaccination recommendations could"

In [16]:
def calculate_rogue(row):
    response = test_inference(row['messages'][0]['content'])
    result = metric.compute(predictions=[response], references=[row['output']], use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result['response']=response
    return result

In [38]:
num_samples=50
prompts = [pipe.tokenizer.apply_chat_template([{"role": "user", "content": dataset_chatml['test'][i]['messages'][0]['content']}], tokenize=False, add_generation_prompt=True)
                                              for i in range(num_samples)]
outputs = pipe(prompts, batch_size=1, max_new_tokens=100, do_sample=True, num_beams=1, temperature=0.7, top_k=50, top_p=0.9,repetition_penalty=1.2,
                   max_time= 180)
# print(outputs)
preds = []
for i in range(len(outputs)):
    generated_text = outputs[i][0]['generated_text']

    response = generated_text[len(prompts[i]):].split()
    # print(response)
    if len(response) > 1:
        # Extract the counter argument
        pred = " ".join(response)
    else:
        pred = ""  # Handle case with no valid split
    preds.append(pred)

    # Print prediction and corresponding reference
    print(f"Prediction {i + 1}: {pred}")
    print(f"Reference {i + 1}: {dataset_chatml['test'][i]['counter_argument']}")
    print("---")  # Separator for clarity
references= [dataset_chatml['test'][i]['counter_argument'] for i in range(len(outputs))]
metric.add_batch(predictions=preds, references=references)
tweets = [dataset_chatml['test'][i]['text'] for i in range(len(outputs))]

# store the tweets predictions and references in a csv file 
import pandas as pd
df = pd.DataFrame(list(zip(tweets, preds, references)), columns =['tweets', 'predictions', 'references'])
df.to_csv('/home/sohampoddar/HDD2/utsav/Predictions_Label_Desc/New_Predictions/phi-3_labels.csv', index=False)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


Prediction 1: While it's true there is ongoing research into all medical interventions, including COVID-19 vaccines like AstraZeneca and Pfizer/BioNTech (Moderna), regulatory bodies such as the FDA conduct rigorous evaluations based on extensive data before approval. Vaccines undergo multiple phases of clinical trials involving tens of thousands of participants over several years prior to authorization—far exceeding standard practices in many fields due to ur
Reference 1: While it's true that the COVID-19 vaccines were developed and authorized quickly, this speed did not come at the expense of safety or efficacy. Regulatory agencies like the FDA and EMA conducted rigorous reviews of extensive clinical trial data before granting emergency use authorization. Additionally, ongoing monitoring and studies continue to assess long-term effects. Historical data shows that vaccines typically provide long-term immunity and safety; vaccines like measles and polio have been extensively studied for

In [18]:
metric.add_batch(predictions=preds, references=references)
result = metric.compute(use_stemmer=True)

In [19]:
import numpy as np

# Assuming result contains your ROUGE scores
rouge1_mean = np.mean(result['rouge1']) * 100
rouge2_mean = np.mean(result['rouge2']) * 100
rougeL_mean = np.mean(result['rougeL']) * 100
rougeLsum_mean = np.mean(result['rougeLsum']) * 100

print("Rouge 1 Mean: ", rouge1_mean)
print("Rouge 2 Mean: ", rouge2_mean)
print("Rouge L Mean: ", rougeL_mean)
print("Rouge Lsum Mean: ", rougeLsum_mean)

Rouge 1 Mean:  37.083042957661064
Rouge 2 Mean:  10.59812928021854
Rouge L Mean:  21.80358370691243
Rouge Lsum Mean:  21.799243769426756


In [24]:
!pip install bert-score

    PyYAML (>=5.1.*)
            ~~~~~~^


In [20]:
from bert_score import score
P, R, F1 = score(preds, references, lang="en", verbose=True)

# Display BERTScore results
print("BERTScore Precision:", P.mean().item())
print("BERTScore Recall:", R.mean().item())
print("BERTScore F1:", F1.mean().item())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 2/2 [00:00<00:00,  3.74it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 23.04it/s]

done in 0.59 seconds, 85.33 sentences/sec
BERTScore Precision: 0.8723220229148865
BERTScore Recall: 0.886738657951355
BERTScore F1: 0.8794540166854858
